In [1]:
import ollama 
import pandas as pd

import csv



In [ ]:
import os
print(os.listdir())


['benchmark_existing_data.ipynb', 'benchmark_ourDatasets.ipynb', 'data', 'ourDatasets']


## evaluation for cti-mcq with llama3.1 and mistral

In [11]:
mcq = pd.read_csv('data/cti-mcq.tsv', sep="\t")


here i'm gonna generate answer from llama3.1 and then evaluate the dataset

In [ ]:
def llm_answer(prompt):
   return  ollama.generate(model='llama3.1', prompt=prompt)['response']

new_data = dict()
new_data['answer'] = list()

limit = 100

for index in range(limit):
   prompt = mcq.iloc[index]['Prompt']
   answer = llm_answer(prompt)
   print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(mcq.index)[:limit])
old_df = mcq.head(limit)

mcq_git = pd.merge(old_df, new_df, left_index=True, right_index=True)

print(mcq_git.head())

mcq_git.to_csv('data/generated_answer/mcq_git.csv', index=False)


here i'm gonna generate answer from mistral and then evaluate the dataset

In [ ]:
def llm_answer(prompt):
   return  ollama.generate(model='mistral', prompt=prompt)['response']

new_data = dict()
new_data['answer'] = list()

limit = 100

for index in range(limit):
   prompt = mcq.iloc[index]['Prompt']
   answer = llm_answer(prompt)
   print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(mcq.index)[:limit])
old_df = mcq.head(limit)

mcq_gitM = pd.merge(old_df, new_df, left_index=True, right_index=True)

print(mcq_gitM.head())

mcq_gitM.to_csv('data/generated_answer/mcq_gitM.csv', index=False)

here i'm gonna do the evaluation

In [8]:
import pandas as pd

df1 = pd.read_csv("data/generated_answer/mcq_git.csv")
df2 = pd.read_csv('data/generated_answer/mcq_gitM.csv')

def extract_predicted_option(s:str):
    options = ['A', 'B', 'C', 'D']
    s = s[::-1] #reverse string, find first uppercase with previous and next character not alphabetic (A*,  A , A.) ...
    for i in range(len(s)):
        if  s[i] in options and (True if i == 0 else not s[i-1].isalpha()) and (True if i == len(s)-1 else not s[i+1].isalpha()): 
            return s[i]
    return ""

def evaluate(df):
    correct_predictions = 0
    for i in range(len(df)):
        gt = df.iloc[i]['GT']
        llm_answer = df.iloc[i]['answer']
        predicted =  extract_predicted_option(llm_answer)
        if predicted == gt:
            correct_predictions+=1
    total_predictions = len(df)
    return correct_predictions / total_predictions * 100


print(evaluate(df1))
print(evaluate(df2))

36.0
44.0


## evaluation cti-rcm with llama3.1 and mistral

In [10]:
rcm = pd.read_csv('data/cti-rcm.tsv', sep="\t")


first im going to generate answer from llama3.1 from the cti-rcm

In [ ]:
def llm_answer(prompt):
   return ollama.generate(model="llama3.1", prompt=prompt, options={"temperature": 0}).get('response', "")

new_data = dict()
new_data['answer'] = list()

limit = 100

for index in range(limit):
   prompt = rcm.iloc[index]['Prompt']
   answer = llm_answer(prompt)
   print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(rcm.index)[:limit])
old_df = rcm.head(limit)

rcm_git = pd.merge(old_df, new_df, left_index=True, right_index=True)
print(rcm_git.head())

rcm_git.to_csv('data/generated_answer/rcm_git.csv', index=False)

now im going to generate answer with mistral 

In [ ]:
def llm_answer(prompt):
   return ollama.generate(model="mistral", prompt=prompt, options={"temperature": 0}).get('response', "")

new_data = dict()
new_data['answer'] = list()

limit = 100

for index in range(limit):
   prompt = rcm.iloc[index]['Prompt']
   answer = llm_answer(prompt)
   print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(rcm.index)[:limit])
old_df = rcm.head(limit)

rcm_git = pd.merge(old_df, new_df, left_index=True, right_index=True)
print(rcm_git.head())

rcm_git.to_csv('data/generated_answer/rcm_gitM.csv', index=False)

now im going to do the evaluation for both mistral and llama 3.1

In [13]:

# Extract the predicted CWE option from LLM response
def extract_predicted_cwe(s: str):
    # Example of a valid CWE pattern could be 'CWE-###' (where ### is a number)
    if 'CWE-' in s:
        start_idx = s.find('CWE-')
        end_idx = start_idx + 5
        while end_idx < len(s) and s[end_idx].isdigit():
            end_idx += 1
        return s[start_idx:end_idx].upper()
    return ""

# General function to evaluate accuracy based on Ground Truth (GT) and answers
def evaluate(df):
    correct_predictions = 0
    for i in range(len(df)):
        gt = df.iloc[i]['GT'].upper()
        llm_answer = df.iloc[i]['answer']
        predicted = extract_predicted_cwe(llm_answer)
        if predicted == gt:
            correct_predictions += 1
    total_predictions = len(df)
    return correct_predictions / total_predictions * 100

# Compute accuracy specifically for RCM dataset using tab-separated values (CWE format)
def compute_rcm_accuracy(fname, col):
    df = pd.read_csv(fname, sep='\t')
    correct = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['GT'].upper()
        if pred.startswith('CWE-'):
            total += 1
            if pred == gt:
                correct += 1
        else:
            print(f'Invalid response at row {idx + 1}')
    return correct / total * 100

# Example usage
df1 = pd.read_csv("data/generated_answer/rcm_git.csv")
df2 = pd.read_csv("data/generated_answer/rcm_gitM.csv")

print("Accuracy for rcm_git.csv:", evaluate(df1))
print("Accuracy for rcm_gitM.csv:", evaluate(df2))



Accuracy for rcm_git.csv: 53.0
Accuracy for rcm_gitM.csv: 46.0


## evaluation for cti-vsp with mistral and llama 3.1

In [4]:
vsp = pd.read_csv('data/cti-vsp.tsv', sep="\t")


first im going to generate answer from llama3.1 from the cti-rcm

In [15]:
#function to generate model answer for vsp (change limit)
def llm_answer(prompt):
   return ollama.generate(model="llama3.1", prompt=prompt, options={"temperature": 0}).get('response', "")


new_data = dict()
new_data['answer'] = list()
start = 0
limit = 300

for index in range(start, limit):
   prompt = vsp.iloc[index]['Prompt']
   print(index)
   answer = llm_answer(prompt)
   #print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(vsp.index)[:limit])
old_df = vsp.head(limit)
final_vsplama = pd.merge(old_df, new_df, left_index=True, right_index=True)
#print(final_rcm.head())
final_vsplama.to_csv('data/generated_answer/vsp/final_vsplama.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

: 

In [5]:
#function to generate model answer for vsp (change limit)
def llm_answer(prompt):
   return ollama.generate(model="mistral", prompt=prompt, options={"temperature": 0}).get('response', "")


new_data = dict()
new_data['answer'] = list()
start = 0
limit = 200

for index in range(start, limit):
   prompt = vsp.iloc[index]['Prompt']
   print(index)
   answer = llm_answer(prompt)
   #print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(vsp.index)[:limit])
old_df = vsp.head(limit)
final_vspM = pd.merge(old_df, new_df, left_index=True, right_index=True)
#print(final_rcm.head())
final_vspM.to_csv('data/generated_answer/vsp/final_vspM.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [7]:

# Load the CSV file
final_vsp = pd.read_csv('data/generated_answer/vsp/final_vspM.csv')

# Example CVSS vector to extract
example = "CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H"

# Initialize an empty dictionary to store the new data
new_data = dict()
new_data['vector'] = list()

# Define limit to control how many rows to process
limit = 200  # Modify the limit as needed

# Loop through each row in the DataFrame
for i in range(len(final_vsp)):
    ans = final_vsp.iloc[i]['answer']
    index = ans.find("CVSS:3.1")  # Find the starting index of the CVSS vector
    if index != -1:  # Ensure the vector is found
        vector = ans[index:index+len(example)]  # Extract the vector
    else:
        vector = None  # Handle cases where no CVSS vector is found
    new_data['vector'].append(vector)

# Create a new DataFrame for the extracted vectors
new_df = pd.DataFrame(new_data, index=final_vsp.index[:limit])

# Select the old data (first 'limit' rows)
old_df = final_vsp.head(limit)

# Merge the old data with the new CVSS vectors
final_vspmistral = pd.merge(old_df, new_df, left_index=True, right_index=True)

# Save the merged DataFrame to a new CSV file
final_vspmistral.to_csv('data/generated_answer/vsp/final_vspM_Vector.csv', index=False)

print("CSV file 'final_vspM_Vector.csv' has been created.")


CSV file 'final_vspM_Vector.csv' has been created.


In [8]:
!pip install cvss
from cvss import CVSS3

In [9]:
def get_cvss_score(cvss_vector):
    c = CVSS3(cvss_vector)
    cvss_score = c.scores()[0]
    return cvss_score

def compute_vsp_mad(fname, col):
    cvss_prefix = ''   # should be empty string if the model responds with the prefix
    #df = pd.read_csv(fname, sep='\t')
    df = final_vspmistral
    error = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['GT'].upper()
        try:
            pred_vector = cvss_prefix + pred
            pred_score = get_cvss_score(pred_vector)
            gt_score = get_cvss_score(gt)
            print(pred_score, gt_score)
            print(abs(pred_score-gt_score))
            error += abs(pred_score-gt_score)
        except Exception as e:
            print('Invalid response at row {}'.format(idx+1))
            print(e)
            continue
        total += 1
    print(error)
    return error/total

In [10]:
compute_vsp_mad("", col = "vector")

7.2 5.5
1.7000000000000002
7.2 8.1
0.8999999999999995
10.0 5.4
4.6
8.4 7.8
0.6000000000000005
9.1 5.4
3.6999999999999993
10.0 5.4
4.6
8.2 7.8
0.39999999999999947
10.0 9.8
0.1999999999999993
8.8 6.1
2.700000000000001
7.5 5.5
2.0
10.0 6.1
3.9000000000000004
10.0 7.8
2.2
7.5 4.4
3.0999999999999996
8.3 8.8
0.5
6.7 7.8
1.0999999999999996
10.0 7.5
2.5
10.0 9.8
0.1999999999999993
10.0 9.8
0.1999999999999993
9.1 9.8
0.7000000000000011
9.6 6.1
3.5
9.6 9.8
0.20000000000000107
9.8 7.2
2.6000000000000005
10.0 5.4
4.6
10.0 7.5
2.5
10.0 9.8
0.1999999999999993
9.1 6.1
3.0
10.0 7.5
2.5
9.8 9.1
0.7000000000000011
10.0 6.5
3.5
10.0 9.8
0.1999999999999993
9.1 5.3
3.8
9.1 6.5
2.5999999999999996
8.3 9.8
1.5
9.8 7.5
2.3000000000000007
9.8 5.5
4.300000000000001
7.2 7.8
0.5999999999999996
9.8 5.5
4.300000000000001
8.8 5.4
3.4000000000000004
9.8 5.3
4.500000000000001
10.0 3.3
6.7
10.0 9.8
0.1999999999999993
7.5 7.5
0.0
7.2 5.5
1.7000000000000002
9.6 9.8
0.20000000000000107
9.0 9.8
0.8000000000000007
9.0 9.8
0.

2.1601010101010107

evaluation for vsp lama 

In [13]:

# Load the CSV file
final_vsp = pd.read_csv('data/generated_answer/vsp/final_vsplama.csv')

# Example CVSS vector to extract
example = "CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H"

# Initialize an empty dictionary to store the new data
new_data = dict()
new_data['vector'] = list()

# Define limit to control how many rows to process
limit = 300  # Modify the limit as needed

# Loop through each row in the DataFrame
for i in range(len(final_vsp)):
    ans = final_vsp.iloc[i]['answer']
    index = ans.find("CVSS:3.1")  # Find the starting index of the CVSS vector
    if index != -1:  # Ensure the vector is found
        vector = ans[index:index+len(example)]  # Extract the vector
    else:
        vector = None  # Handle cases where no CVSS vector is found
    new_data['vector'].append(vector)

# Create a new DataFrame for the extracted vectors
new_df = pd.DataFrame(new_data, index=final_vsp.index[:limit])

# Select the old data (first 'limit' rows)
old_df = final_vsp.head(limit)

# Merge the old data with the new CVSS vectors
final_vspllama = pd.merge(old_df, new_df, left_index=True, right_index=True)

# Save the merged DataFrame to a new CSV file
final_vspllama.to_csv('data/generated_answer/vsp/final_vspL_vector.csv', index=False)

print("CSV file 'final_vspL_vector' has been created.")


CSV file 'final_vspL_vector' has been created.


In [14]:
def get_cvss_score(cvss_vector):
    c = CVSS3(cvss_vector)
    cvss_score = c.scores()[0]
    return cvss_score

def compute_vsp_madlama(fname, col):
    cvss_prefix = ''   # should be empty string if the model responds with the prefix
    #df = pd.read_csv(fname, sep='\t')
    df = final_vspllama
    error = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['GT'].upper()
        try:
            pred_vector = cvss_prefix + pred
            pred_score = get_cvss_score(pred_vector)
            gt_score = get_cvss_score(gt)
            print(pred_score, gt_score)
            print(abs(pred_score-gt_score))
            error += abs(pred_score-gt_score)
        except Exception as e:
            print('Invalid response at row {}'.format(idx+1))
            print(e)
            continue
        total += 1
    print(error)
    return error/total

In [15]:
compute_vsp_madlama("", col = "vector")

9.8 5.5
4.300000000000001
4.3 8.1
3.8
8.6 5.4
3.1999999999999993
7.7 7.8
0.09999999999999964
10.0 5.4
4.6
5.4 5.4
0.0
0.0 7.8
7.8
6.5 9.8
3.3000000000000007
6.5 6.1
0.40000000000000036
7.5 5.5
2.0
8.1 6.1
2.0
9.6 7.8
1.7999999999999998
4.4 4.4
0.0
10.0 8.8
1.1999999999999993
6.8 7.8
1.0
7.5 7.5
0.0
9.8 9.8
0.0
8.6 9.8
1.200000000000001
7.3 9.8
2.500000000000001
10.0 6.1
3.9000000000000004
9.6 9.8
0.20000000000000107
8.8 7.2
1.6000000000000005
9.8 5.4
4.4
8.6 7.5
1.0999999999999996
8.6 9.8
1.200000000000001
9.4 6.1
3.3000000000000007
9.4 7.5
1.9000000000000004
10.0 9.1
0.9000000000000004
8.6 6.5
2.0999999999999996
9.8 9.8
0.0
9.3 5.3
4.000000000000001
9.4 6.5
2.9000000000000004
10.0 9.8
0.1999999999999993
7.5 7.5
0.0
8.2 5.5
2.6999999999999993
8.4 7.8
0.6000000000000005
8.2 5.5
2.6999999999999993
7.2 5.4
1.7999999999999998
8.2 5.3
2.8999999999999995
5.8 3.3
2.5
9.8 9.8
0.0
8.8 7.5
1.3000000000000007
8.6 5.5
3.0999999999999996
10.0 9.8
0.1999999999999993
9.8 9.8
0.0
8.8 9.8
1.0
10.0 9.8


1.763210702341136

## generation of the summary 

In [8]:
import pandas as pd
from cvss import CVSS3

# Chargement des fichiers d'évaluation
mcq_llama = pd.read_csv('data/generated_answer/mcq_git.csv')
mcq_mistral = pd.read_csv('data/generated_answer/mcq_gitM.csv')
rcm_llama = pd.read_csv('data/generated_answer/rcm_git.csv')
rcm_mistral = pd.read_csv('data/generated_answer/rcm_gitM.csv')
vsp_llama = pd.read_csv('data/generated_answer/vsp/final_vspL_vector.csv')
vsp_mistral = pd.read_csv('data/generated_answer/vsp/final_vspM_Vector.csv')

# Fonction evaluate pour les évaluations MCQ
def evaluate(df):
    correct_predictions = 0
    for i in range(len(df)):
        gt = df.iloc[i]['GT']
        llm_answer = df.iloc[i]['answer']
        predicted = extract_predicted_option(llm_answer)
        if predicted == gt:
            correct_predictions += 1
    total_predictions = len(df)
    return correct_predictions / total_predictions * 100

# Fonction extract_predicted_option pour extraire la réponse prédite
def extract_predicted_option(s: str):
    options = ['A', 'B', 'C', 'D']
    s = s[::-1]  # Inverse la chaîne pour trouver la première lettre
    for i in range(len(s)):
        if s[i] in options and (i == 0 or not s[i - 1].isalpha()) and (i == len(s) - 1 or not s[i + 1].isalpha()):
            return s[i]
    return ""

# Calcul des scores pour chaque évaluation MCQ
mcq_llama_accuracy = evaluate(mcq_llama)
mcq_mistral_accuracy = evaluate(mcq_mistral)

# Calcul des scores pour les RCM en comparant les réponses avec la vérité terrain
rcm_llama_accuracy = (rcm_llama['answer'] == rcm_llama['GT']).mean() * 100
rcm_mistral_accuracy = (rcm_mistral['answer'] == rcm_mistral['GT']).mean() * 100

# Fonction get_cvss_score (déjà définie dans votre code)
def get_cvss_score(cvss_vector):
    c = CVSS3(cvss_vector)
    return c.scores()[0]

# Calcul des MAD pour VSP
def compute_vsp_madlama(fname, col='vector'):
    df = vsp_llama
    error = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row[col].upper()  # Vecteur prédictif
        gt = row['GT'].upper()  # Vecteur attendu
        try:
            pred_score = get_cvss_score(pred)
            gt_score = get_cvss_score(gt)
            error += abs(pred_score - gt_score)
        except Exception as e:
            print(f'Invalid response at row {idx+1}: {e}')
            continue
        total += 1
    return error / total

def compute_vsp_mad(fname, col='vector'):
    df = vsp_mistral
    error = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['GT'].upper()
        try:
            pred_score = get_cvss_score(pred)
            gt_score = get_cvss_score(gt)
            error += abs(pred_score - gt_score)
        except Exception as e:
            print(f'Invalid response at row {idx+1}: {e}')
            continue
        total += 1
    return error / total

# Calcul des MAD pour VSP
vsp_llama_mad = compute_vsp_madlama("")
vsp_mistral_mad = compute_vsp_mad("")

# Création d'un DataFrame pour résumer les résultats
results = {
    "Model": ["LLaMA", "Mistral"],
    "MCQ Accuracy (%)": [mcq_llama_accuracy, mcq_mistral_accuracy],
    "RCM Accuracy (%)": [rcm_llama_accuracy, rcm_mistral_accuracy],
    "VSP MAD (Mean Absolute Difference)": [vsp_llama_mad, vsp_mistral_mad]
}

summary_df = pd.DataFrame(results)

# Sauvegarde des résultats dans un fichier CSV
summary_df.to_csv('evaluation_summary.csv', index=False)

print("Les résultats des évaluations ont été enregistrés dans 'evaluation_summary.csv'.")
print(summary_df)


Invalid response at row 101: Unknown value "M" in field "C:M"
Invalid response at row 168: Unknown value "[L OR" in field "I:[L OR"
Invalid response at row 194: Unknown value "A" in field "PR:A"
Les résultats des évaluations ont été enregistrés dans 'evaluation_summary.csv'.
     Model  MCQ Accuracy (%)  RCM Accuracy (%)  \
0    LLaMA              36.0               0.0   
1  Mistral              44.0               0.0   

   VSP MAD (Mean Absolute Difference)  
0                            1.763211  
1                            2.160101  


In [7]:
print(vsp_llama.columns)
print(vsp_mistral.columns)


Index(['URL', 'Description', 'Prompt', 'GT', 'answer', 'vector'], dtype='object')
Index(['URL', 'Description', 'Prompt', 'GT', 'answer', 'vector'], dtype='object')
